In [ ]:
# To add / remove sports, update the following:

# Script modes - add / remove
# Weeks - comment / uncomment
# EVs - comment / uncomment the sports
# Update the def get_mode_params to add / remove sports

In [ ]:
!pip install py_clob_client
!pip install schedule

import subprocess
import csv

try:
    subprocess.run(['pip', 'install', 'py_clob_client'], check=True)
    subprocess.run(['pip', 'install', 'schedule'], check=True)
except subprocess.CalledProcessError as e:
    print(f"Error installing packages: {e}")

In [ ]:
#### SETTINGS ####

LOG_LEVEL_ENV = 'Disabled'  # Options are DISABLED (preferred) | DEBUG (Only while debugging) | INFO (in between)

# Pull in your Polymarket key and funder values, as well as your Pinnacle Odds API key from a CSV file
# The CSV file should have the following columns: key, funder, X_RAPID_API_KEY

csv_file = "gambot_keys.csv"
with open(csv_file, newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip header row
    first_row = next(reader)  # Read the first data row
    
    # Assign values to variables
    key = first_row[0]
    funder = first_row[1]
    X_RAPID_API_KEY = first_row[2]

# SCRIPT_MODES = ["NBA","NFL","NHL"]
SCRIPT_MODES = ["NBA","NHL"]

INCLUDE_LIVE = "False"  # Options are "True" or "False"

# Validate INCLUDE_LIVE setting
if INCLUDE_LIVE.lower() not in ["true", "false"]:
    logger.warning(f"Invalid INCLUDE_LIVE value: {INCLUDE_LIVE}. Defaulting to excluding live games.")

# Put guardrails around the shortest and longest odds that Gambot will bet on
MAXIMUM_PINNACLE_TRUE_PROBABILITY = 0.95
MINIMUM_PINNACLE_TRUE_PROBABILITY = 0.05

# Put guardrails around bet sizing
MAX_PERCENTAGE_BET = 10 # This is as a percent of bankroll
MAX_DOLLAR_BET = 50

time_interval = 0  # Set the break time after each cycle the code runs. Default is zero, but you may want to set this according to your API pulls.

nba_week=24 #Set Sunday night
#nfl_week=23 #Set xxx night
nhl_week=27 #set Saturday night

# Format: [NBA, NFL, NHL, EPL]
# week = [nba_week, nfl_week, nhl_week]
# kelly_multiplier = [0.5, 0.5, 0.5]
# target_buy_ev = [0.03, 0.03, 0.03]
# target_sell_ev = [0.02, 0.02, 0.02]

week = [nba_week, nhl_week]
kelly_multiplier = [0.5, 0.5]
target_buy_ev = [0.03, 0.03]
target_sell_ev = [0.99, 0.99]

# When trading will start and end in military time
trading_start=0
trading_end=24

### IMPORTS ###

import datetime
from datetime import datetime
import pytz
import math
import logging
import requests
import json
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import BookParams, AssetType, BalanceAllowanceParams, TradeParams, OrderType, OrderArgs
from py_clob_client.constants import POLYGON
from py_clob_client.exceptions import PolyApiException
from py_clob_client.order_builder.constants import BUY, SELL
from collections import defaultdict as df
import time

class Logger:
    @staticmethod
    def setup_logger(logger_name: str = 'gambot'):
        LOG_LEVEL = LOG_LEVEL_ENV.upper()
        logger = logging.getLogger(logger_name)

        # If logging is disabled, set it up before disabling
        if LOG_LEVEL == 'DISABLED':
            logger.setLevel(logging.CRITICAL)  # Set to CRITICAL before disabling
            logger.disabled = True
            # Remove any existing handlers
            for handler in logger.handlers[:]:
                logger.removeHandler(handler)
            return logger

        # Only proceed with handler setup if logging is enabled
        if LOG_LEVEL == 'DEBUG':
            logger.setLevel(logging.DEBUG)
        else:
            logger.setLevel(logging.INFO)

        # Create a console handler and set the level
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.DEBUG if LOG_LEVEL == 'DEBUG' else logging.INFO)

        # Create a formatter and add it to the handler
        formatter = logging.Formatter(f'%(asctime)s - %(levelname)s - {logger_name} - %(message)s')
        console_handler.setFormatter(formatter)

        # Remove any existing handlers before adding new one
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)

        # Add the handler to the logger
        logger.addHandler(console_handler)

        return logger


logger = Logger.setup_logger()

ODDS_API_URL = "https://pinnacle-odds.p.rapidapi.com/kit/v1"

SINCE = [None for _ in range(len(SCRIPT_MODES))]

headers = {
    "x-rapidapi-key": X_RAPID_API_KEY,
    "x-rapidapi-host": "pinnacle-odds.p.rapidapi.com"
}

GLOBAL_PINNACLE_MATCHUPS = [{} for _ in range(len(SCRIPT_MODES))]

def get_mode_params(script_mode):
    # Returns in the format (pinnacle_params, poly_params)
    # if script_mode == "NFL":
    #     return ({
    #                 "sport_id": "7",
    #                 "is_have_odds": "true",
    #                 "league_ids": "889",
    #             }, {
    #                 'series_id': '1',
    #                 'closed': 'false',
    #                 'event_week': nfl_week,
    #                 'series_closed': 'true'
    #             })
    # elif script_mode == "NBA":
    if script_mode == "NBA":
        return ({
                    "sport_id": "3",
                    "is_have_odds": "true",
                    "league_ids": "487"
                }, {
                    'series_id': '2',
                    'closed': 'false',
                    'event_week': nba_week,
                    'series_closed': 'true'
                })
    elif script_mode == "NHL":
        return ({
                    "sport_id": "4",
                    "is_have_odds": "true",
                    "league_ids": "1456",
                }, {
                    'series_id': '4',
                    'closed': 'false',
                    'event_week':nhl_week,
                    'series_closed': 'true',
                })                  

    else:
        raise ValueError(f"Unsupported SCRIPT_MODE: {script_mode}")

def pull_from_pinnacle(params):
    global SINCE
    global GLOBAL_PINNACLE_MATCHUPS
    try:
        logger.info("Fetching data from Pinnacle API...")  # Log the beginning of the process
        if SINCE[ind] is not None:
            params["since"] = SINCE[ind]
        # print(params)
        response = requests.get(f"{ODDS_API_URL}/markets", params=params, headers=headers)
        response = response.json()

        SINCE[ind] = response["last_call"]

        # print("PINNACLE API response:", response)
        logger.debug(f"API response received: {response}")  # Log the raw API response for debugging

        events = response["events"]
        logger.info(f"Processing {len(events)} events...")  # Log the number of events received
        
        GLOBAL_PINNACLE_MATCHUPS_TEMP = {}
        for event in events:
            # Check live game filtering first
            event_type = event.get('event_type', 'prematch')
            if INCLUDE_LIVE.lower() == "false" and event_type == 'live':
                logger.info(f"Skipping live game: {event['home']} vs {event['away']}")
                continue
            
            logger.debug(f"Processing event: {event['home']} vs {event['away']}")
            
            try:
                period_data = event['periods']['num_0']
                
                # According to Pinnacle API docs, a market is open for betting if:
                # 1. Period status is 1 or O
                # 2. Market (period) has odds
                # 3. Period has cutoff in the future
                
                # Ensure period status is open for betting
                period_status = str(period_data.get('period_status'))
                if period_status not in ['1', 'O']:
                    logger.info(f"Skipping {event['home']} vs {event['away']} - period not open for betting (status = {period_status})")
                    continue

                # Check if market has odds and is not suspended
                if not period_data.get('money_line'):
                    logger.info(f"Skipping {event['home']} vs {event['away']} - no odds available")
                    continue


                # Check cutoff time
                if 'cutoff' not in period_data:
                    logger.info(f"Skipping {event['home']} vs {event['away']} - no cutoff time")
                    continue
                
                # Make cutoff time timezone-aware (API uses GMT/UTC)
                cutoff_time = datetime.fromisoformat(period_data['cutoff'])
                if cutoff_time.tzinfo is None:
                    cutoff_time = pytz.UTC.localize(cutoff_time)
                
                current_time = datetime.now(pytz.UTC)
                
                if cutoff_time <= current_time:
                    logger.info(f"Skipping {event['home']} vs {event['away']} - cutoff time passed")
                    continue

                # Validate odds data
                try:
                    home_odds = float(period_data['money_line']['home'])
                    away_odds = float(period_data['money_line']['away'])
                    if home_odds <= 0 or away_odds <= 0:
                        logger.warning(f"Invalid odds for {event['home']} vs {event['away']}: {home_odds}, {away_odds}")
                        continue
                except (ValueError, TypeError) as e:
                    logger.error(f"Error parsing odds for {event['home']} vs {event['away']}: {str(e)}")
                    continue

                # Make game start time timezone-aware
                game_time = datetime.fromisoformat(event["starts"])
                if game_time.tzinfo is None:
                    game_time = pytz.UTC.localize(game_time)

                # Store game data only if all checks pass
                GLOBAL_PINNACLE_MATCHUPS_TEMP[(event["home"], event["away"])] = {
                    "teams": [event["home"], event["away"]],
                    "odds": [home_odds, away_odds],
                    "game_time": game_time.astimezone(pytz.UTC),
                    "is_live": event_type == 'live',
                    "event_type": event_type
                }
                logger.debug(f"Matchup added: {GLOBAL_PINNACLE_MATCHUPS_TEMP[(event['home'], event['away'])]}")

            except KeyError as e:
                logger.error(f"Error processing event {event['home']} vs {event['away']}: Missing key {str(e)}")
                continue
            except Exception as e:
                logger.error(f"Error processing event {event['home']} vs {event['away']}: {str(e)}")
                continue

        # Update global matchups
        GLOBAL_PINNACLE_MATCHUPS[ind] = GLOBAL_PINNACLE_MATCHUPS_TEMP

        logger.info(f"Finished processing events. Total matchups: {len(GLOBAL_PINNACLE_MATCHUPS[ind])}")
        return GLOBAL_PINNACLE_MATCHUPS[ind].values()

    except requests.exceptions.RequestException as e:
        logger.error(f"Network error while fetching data from Pinnacle: {str(e)}")
        raise Exception("Network error while fetching data from Pinnacle")
    except KeyError as e:
        logger.error(f"KeyError: {str(e)}")
        raise Exception("No data to pull from Pinnacle")
    except json.JSONDecodeError as e:
        logger.error(f"Error decoding JSON response from Pinnacle: {str(e)}")
        raise Exception("Invalid JSON response from Pinnacle")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        raise e

def calc_true_odds(odds):
    odd_a, odd_b = odds

    implied_probability_a = 1 / odd_a
    implied_probability_b = 1 / odd_b

    total_implied_probability = implied_probability_a + implied_probability_b

    true_probability_a = implied_probability_a / total_implied_probability
    true_probability_b = implied_probability_b / total_implied_probability

    true_decimal_odds_a = 1 / true_probability_a
    true_decimal_odds_b = 1 / true_probability_b

    return (true_decimal_odds_a, true_decimal_odds_b), (true_probability_a, true_probability_b)


def calc_expected_value(selected_teams):
    for team in selected_teams:
        logger.debug(f"Processing team: {team['team']}")  # Logging team information
        expected_value = ((team['pinnacle_true_probability'] * team['poly_odds']) - 1) * 100
        team['expected_value'] = expected_value
        logger.debug(f"Calculated expected value for team {team['team']}: {expected_value}")  # Logging expected value


def calc_kelly_optimal_and_adjusted_bet_size(selected_teams, bankroll, kelly_multiplier):
    for team in selected_teams:
        logger.debug(f"Calculating Kelly bet size for team: {team['team']}")  # Logging bet size calculation start

        f_star = ((team['pinnacle_true_probability'] * (team['poly_odds'] - 1)) - (
                1 - team['pinnacle_true_probability'])) / (team['poly_odds'] - 1)

        if f_star < 0:
            logger.debug(f"Skipping team {team['team']} as f_star is negative.")  # Logging negative f_star
            continue

        team['kelly_optimal_bet_size'] = f_star * bankroll
        team['kelly_adjusted_bet_size'] = team['kelly_optimal_bet_size'] * kelly_multiplier
        logger.debug(f"Kelly optimal bet size for team {team['team']}: {team['kelly_optimal_bet_size']}")
        logger.debug(f"Kelly adjusted bet size for team {team['team']}: {team['kelly_adjusted_bet_size']}")


def calc_target_variables(true_probability, target_buy_ev, target_sell_ev, bankroll, kelly_multiplier, poly_odds):
    buy_target = true_probability * (1 - target_buy_ev)
    sell_target = true_probability * (1 + target_sell_ev)
    f_star = true_probability - ((1 - true_probability) / ((1 / buy_target) - 1))
    target_kelly_optimal_bet_size = f_star * bankroll
    target_kelly_adjusted_bet_size = target_kelly_optimal_bet_size * kelly_multiplier
    # print(true_probability, target_buy_ev, target_sell_ev, bankroll, kelly_multiplier)
    # print(buy_target, sell_target, f_star)
    return buy_target, sell_target, target_kelly_optimal_bet_size, target_kelly_adjusted_bet_size

def merge_and_process_data(polymarket_data, pinnacle_data, user_inputs=None, for_trade=False):
    initial_processed_data = []
    trading_data = []
    current_time=datetime.now(pytz.utc)

    # Print header once at the start
    header = f"\n{'Team':25} | {'Buy Price':10} | {'True Prob':10} | {'Premium':10}"
    divider = "-" * 61
    logger.info(divider)
    logger.info(header)
    logger.info(divider)
    # Comment this out if you don't want the price vs. true probability

    for poly_teams in polymarket_data:
        poly_team_1, poly_team_2 = poly_teams['teams']
        for pinnacle_teams in pinnacle_data:
            # Skip if this is a live game and we don't want live games
            if INCLUDE_LIVE.lower() == "false" and pinnacle_teams.get('is_live', False):
                logger.info(f"Skipping live game in merge: {poly_team_1} vs {poly_team_2}")
                continue
               
            pinnacle_team_1, pinnacle_team_2 = pinnacle_teams['teams']
            # Make poly_game_time timezone-aware if it isn't already
            poly_game_time = poly_teams["game_time"]
            if poly_game_time.tzinfo is None:
                poly_game_time = pytz.UTC.localize(poly_game_time)
            
            pinnacle_game_time = pinnacle_teams["game_time"]
            if pinnacle_game_time.tzinfo is None:
                pinnacle_game_time = pytz.UTC.localize(pinnacle_game_time)
            
            if poly_game_time.date() == pinnacle_game_time.date():
                if poly_team_1 in pinnacle_team_1 and poly_team_2 in pinnacle_team_2:
                    pass
                elif poly_team_1 in pinnacle_team_2 and poly_team_2 in pinnacle_team_1:
                    pinnacle_teams['teams'] = pinnacle_teams['teams'][::-1]
                    pinnacle_teams['odds'] = pinnacle_teams['odds'][::-1]
                else:
                    continue
            else:
                continue

            pinnacle_true_odds, pinnacle_true_probabilities = calc_true_odds(pinnacle_teams['odds'])

# This piece of the logger prints out the price vs. true probability
            premium1 = ((poly_teams['polymarket_probability_to_buy'][0] - pinnacle_true_probabilities[0]) / pinnacle_true_probabilities[0]) * 100
            premium2 = ((poly_teams['polymarket_probability_to_buy'][1] - pinnacle_true_probabilities[1]) / pinnacle_true_probabilities[1]) * 100
            
            # Pretty print with aligned columns
            logger.info(f"{poly_team_1:25} | {poly_teams['polymarket_probability_to_buy'][0]:10.3f} | {pinnacle_true_probabilities[0]:10.3f} | {premium1:+9.1f}%")
            logger.info(f"{poly_team_2:25} | {poly_teams['polymarket_probability_to_buy'][1]:10.3f} | {pinnacle_true_probabilities[1]:10.3f} | {premium2:+9.1f}%")
            logger.info(divider)
# You can comment out the above logger to not print out the price vs. true probability

            logger.debug(f"Polymarket teams: {poly_teams['teams']}, Pinnacle teams: {pinnacle_teams['teams']}")
            logger.debug(
                f"Pinnacle True Odds: {pinnacle_true_odds}, Pinnacle True Probabilities: {pinnacle_true_probabilities}")

            if for_trade:
                for ind, team in enumerate(poly_teams['teams']):
                    if pinnacle_true_probabilities[ind] < MINIMUM_PINNACLE_TRUE_PROBABILITY or \
                            pinnacle_true_probabilities[ind] > MAXIMUM_PINNACLE_TRUE_PROBABILITY:
                        logger.info(
                            f"Skipping {poly_teams['teams']} as the pinnacle_true_probability: {pinnacle_true_probabilities[ind]} is not in the range {(MINIMUM_PINNACLE_TRUE_PROBABILITY, MAXIMUM_PINNACLE_TRUE_PROBABILITY)}")
                        continue
                    buy_target, sell_target, target_kelly_optimal_bet_size, target_kelly_adjusted_bet_size = calc_target_variables(
                        true_probability=pinnacle_true_probabilities[ind], target_buy_ev=user_inputs['target_buy_ev'],
                        target_sell_ev=user_inputs['target_sell_ev'], bankroll=user_inputs['bankroll'],
                        kelly_multiplier=user_inputs['kelly_multiplier'],
                        poly_odds=poly_teams['polymarket_odds_to_buy'][ind]
                    )

                    trading_data.append({
                        'team': team,
                        'matchups': poly_teams['matchups'],
                        'buy_target': buy_target,
                        'sell_target': sell_target,
                        'target_kelly_optimal_bet_size': target_kelly_optimal_bet_size,
                        'target_kelly_adjusted_bet_size': target_kelly_adjusted_bet_size,
                        'token': poly_teams['tokens'][ind],
                        'condition_id': poly_teams['condition_id'],
                        'polymarket_probability_to_buy': poly_teams['polymarket_probability_to_buy'][ind],
                        'polymarket_probability_to_sell': poly_teams['polymarket_probability_to_sell'][ind],
                        'order_min_size': poly_teams['order_min_size'],
                        'pinnacle_true_probability': pinnacle_true_probabilities[ind],
                        'polymarket_odds_to_buy': poly_teams['polymarket_odds_to_buy'][ind],
                        'polymarket_odds_to_sell': poly_teams['polymarket_odds_to_sell'][ind],
                        'bankroll': user_inputs['bankroll']
                    })

            else:
                if poly_teams['polymarket_odds_to_buy'][0] > pinnacle_true_odds[0]:
                    initial_processed_data.append({
                        'team': poly_team_1,
                        'matchups': poly_teams['matchups'],
                        'poly_odds': poly_teams['polymarket_odds_to_buy'][0],
                        'poly_probability': poly_teams['polymarket_probability_to_buy'][0],
                        'pinnacle_odds': pinnacle_teams['odds'][0],
                        'pinnacle_true_odds': pinnacle_true_odds[0],
                        'pinnacle_true_probability': pinnacle_true_probabilities[0]
                    })
                    logger.debug(f"Added team {poly_team_1} with Polymarket odds higher than Pinnacle true odds.")
                else:
                    logger.debug(f"Skipping team {poly_team_1} with Polymarket odds lower than Pinnacle true odds.")
                if poly_teams['polymarket_odds_to_buy'][1] > pinnacle_true_odds[1]:
                    initial_processed_data.append({
                        'team': poly_team_2,
                        'matchups': poly_teams['matchups'],
                        'poly_odds': poly_teams['polymarket_odds_to_buy'][1],
                        'poly_probability': poly_teams['polymarket_probability_to_buy'][1],
                        'pinnacle_odds': pinnacle_teams['odds'][1],
                        'pinnacle_true_odds': pinnacle_true_odds[1],
                        'pinnacle_true_probability': pinnacle_true_probabilities[1]
                    })
                    logger.debug(f"Added team {poly_team_2} with Polymarket odds higher than Pinnacle true odds.")
                else:
                    logger.debug(f"Skipping team {poly_team_2} with Polymarket odds lower than Pinnacle true odds.")

    if for_trade:
        return trading_data
    return initial_processed_data


def calc_polymarket_decimal_odds(odds):
    odds_a, odds_b = odds
    odds_a, odds_b = 1 / odds_a, 1 / odds_b
    logger.debug(f"Converted Polymarket odds to decimal: {odds_a}, {odds_b}")  # Logging odds conversion
    return odds_a, odds_b


GAMMA_API_URL = 'https://gamma-api.polymarket.com'
host = "https://clob.polymarket.com"

chain_id = POLYGON

# client = ClobClient(host)
client = ClobClient(
    host,
    key=key,
    chain_id=POLYGON,
    funder=funder,
    signature_type=1,
)

client.set_api_creds(client.create_or_derive_api_creds())


def strategy_v1_stage_2(tokens, client: ClobClient):
    logger.debug(f"Processing tokens: {tokens}")  # Logging token list
    params = [BookParams(token_id=token, side=side) for side in ['SELL', 'BUY'] for token in tokens]
    resp = client.get_prices(params=params)
    logger.debug(f"Response from client.get_prices: {resp}")  # Log the response
    return resp


def strategy_v1(response) -> list[dict]:
    # Create CLOB client and get/set API credentials
    matchups = []
    temp_matchups = []
    all_tokens = []
    logger.info(f"Processing {len(response)} events...")  # Log number of events

    for event in response:
        market = event['markets'][0]
        tokens = market['clobTokenIds'].lstrip("[").rstrip("]").replace(" ", "").replace('"', "").split(',')
        outcomes = market['outcomes'].lstrip("[").rstrip("]").replace(" ", "").replace('"', "").split(',')
        all_tokens.extend(tokens)
        temp_matchups.append({
            "teams": outcomes,
            "tokens": tokens,
            "condition_id": market["conditionId"],
            "matchups": event['title'],
            "order_min_size": market['orderMinSize'],
            "game_time": datetime.strptime(market["gameStartTime"][:10], "%Y-%m-%d")
        })

    stage2_response = strategy_v1_stage_2(all_tokens, client)
    # print(all_tokens)
    # print(stage2_response)
    logger.debug(f"Stage 2 response: {stage2_response}")  # Log stage 2 response

    for data in temp_matchups:
        tokens = data['tokens']
        outcomes = data['teams']
        try:
            # Get the polymarket_odds_to_buy for the tokens
            polymarket_odds_to_buy = [stage2_response[token]['SELL'] for token in tokens]
            logger.debug(f"Odds for tokens {tokens}: {polymarket_odds_to_buy}")  # Log polymarket_odds_to_buy
            # print(data["teams"], polymarket_odds_to_buy)

            # Get the polymarket_odds_to_buy for the tokens
            polymarket_odds_to_sell = [stage2_response[token]['BUY'] for token in tokens]
            logger.debug(f"Odds for tokens {tokens}: {polymarket_odds_to_sell}")  # Log polymarket_odds_to_buy
        except KeyError:
            logger.error(f"Error while getting Buy and SELL data for {outcomes}")
            continue
        try:
            matchups.append({
                "teams": outcomes,
                "polymarket_odds_to_buy": [1 / float(odd) for odd in polymarket_odds_to_buy],
                "polymarket_probability_to_buy": [float(odd) for odd in polymarket_odds_to_buy],
                "polymarket_odds_to_sell": [1 / float(odd) for odd in polymarket_odds_to_sell],
                "polymarket_probability_to_sell": [float(odd) for odd in polymarket_odds_to_sell],
                # "polymarket_clob_tokens": tokens,
                "matchups": data['matchups'],
                "condition_id": data['condition_id'],
                "order_min_size": data['order_min_size'],
                "tokens": tokens,
                "game_time": data["game_time"]
            })
        except ZeroDivisionError:
            logger.error(f"{outcomes}, game has ended")
            continue
    return matchups

def pull_from_polymarket_events_api(params, week):
    params['event_week'] = str(week)
    # parse_data = (strategy_v1, strategy_v2)[params_version]
    parse_data = strategy_v1

    logger.info(f"Pulling data from Polymarket API with params: {params}")  # Log API request parameters

    try:
        response = requests.get(f"{GAMMA_API_URL}/events", params=params)
        response = response.json()
        # print("Response from Polymarket API", response)
        logger.debug(f"API response: {response}")  # Log API response
    except Exception as e:  # NOQA
        logger.error(f"Error while fetching data: {str(e)}")
        # print(f"Error while fetching data: {str(e)}")
        raise e

    if len(response) == 0:
        logger.warning("No data available to pull from Polymarket")  # Log empty response
        raise Exception("No data available to pull from Polymarket")

    return parse_data(response)


def pull_user_balance():
    logger.info("Fetching user balance...")
    try:
        account_balance = float(client.get_balance_allowance(
            params=BalanceAllowanceParams(asset_type=AssetType.COLLATERAL)
        )['balance'])
        # print(account_balance)
        logger.debug(f"User balance retrieved: {account_balance}")
        return account_balance / 1e6
    except Exception as e:
        logger.error(f"Error fetching user balance: {str(e)}")
        # print(f"Error fetching user balance: {str(e)}")
        raise e


def pull_user_portfolio(token_ids):
    logger.info(f"Fetching user portfolio for token IDs: {token_ids}")
    portfolio = df(lambda: df(int))
    try:
        for token_id in token_ids:
            logger.debug(f"Fetching trades for token ID: {token_id}")
            portfolio_raw = client.get_trades(
                TradeParams(
                    asset_id=token_id
                )
            )
            for trade in portfolio_raw:
                portfolio[trade["asset_id"]]['amount'] += (float(trade["size"]) * float(trade["price"])) * (1, -1)[
                    trade["side"] == "SELL"]
                portfolio[trade["asset_id"]]['shares'] += float(trade["size"]) * (1, -1)[trade["side"] == "SELL"]
                logger.debug(f"Updated portfolio for asset ID {trade['asset_id']}: {portfolio[trade['asset_id']]}")
    except Exception as e:
        logger.error(f"Error fetching portfolio data: {str(e)}")
        # print(f"Error fetching portfolio data: {str(e)}")
        raise e
    return portfolio


def buying_shares(processed_data):
    hour_of_day = datetime.now(pytz.timezone('US/Eastern')).hour
    if hour_of_day >= trading_start and hour_of_day < trading_end:
        logger.info("Starting buying shares process...")
        buying_stage_1 = []
        token_ids = set()
        for data in processed_data:
            if data["polymarket_probability_to_buy"] < data["buy_target"]:
                buying_stage_1.append(data)
                token_ids.add(data["token"])
                logger.debug(f"Added to buying stage 1: {data}")

        # print(token_ids)
        current_positions = pull_user_portfolio(token_ids=token_ids)

        # print(current_positions)

        logger.debug(f"Current positions for tokens: {current_positions}")

        for data in buying_stage_1:
            amount_to_purchase = max(0, data['target_kelly_adjusted_bet_size'] - current_positions[data['token']]['amount'])
            logger.debug(f"Amount to purchase for token {data['team']} {data['token']}: {amount_to_purchase}")
            print(
                f"Candidate: ${round(amount_to_purchase,2)} or {round(amount_to_purchase / data['polymarket_probability_to_buy'], 1)} shares of {data['team']} at {round(data['polymarket_probability_to_buy'], 2)}. min_order_size is {data['order_min_size']}. Token reference: {data['token']}")
            if amount_to_purchase != 0:
                # print(data['polymarket_probability_to_buy'],
                #       round(amount_to_purchase / data['polymarket_probability_to_buy'], 2))
                if round(amount_to_purchase / data['polymarket_probability_to_buy'], 1) < data['order_min_size']:
                    logger.info(
                        f"Attempted block size is less than the minimum for {data['team']}, we want to buy {round(amount_to_purchase / data['polymarket_probability_to_buy'], 1)}, but min_order_quantity is {data['order_min_size']}")
                    continue
                if amount_to_purchase > data['bankroll'] * (MAX_PERCENTAGE_BET / 100):
                    logger.info(
                        f"Skipping purchase for {data['team']} as the amount_to_purchase {amount_to_purchase} exceeds {MAX_PERCENTAGE_BET}% of bankroll, i.e {data['bankroll'] * (MAX_PERCENTAGE_BET / 100)}")
                    continue
                if amount_to_purchase > MAX_DOLLAR_BET:
                    logger.info(
                        f"Skipping purchase for {data['team']} as the amount_to_purchase {amount_to_purchase} is greater than MAX_DOLLAR_BET: ${MAX_DOLLAR_BET}")
                    continue
                # logger.info(
                #     f"Purchasing {round(amount_to_purchase / data['polymarket_probability_to_buy'], 1)} shares for {data['team']}")
                print(
                    f"Checks passed! Attempting to buy ${round(amount_to_purchase,2)} or {round(amount_to_purchase / data['polymarket_probability_to_buy'], 1)} shares of {data['team']} at {round(data['polymarket_probability_to_buy'], 2)}. min_order_size is {data['order_min_size']}. Token reference: {data['token']}")
                signed_order = client.create_order(
                    order_args=OrderArgs(
                        price=math.floor(data['polymarket_probability_to_buy']*100)/100,
                        #price=data['polymarket_probability_to_buy'],
                        size=round(amount_to_purchase / data['polymarket_probability_to_buy']),
                        side=BUY,
                        #expiration=int(time.time()) + 60 + 60,  # A buffer of 60 seconds to fill as much as possible
                        token_id=data["token"]
                    )
                )
                try:
                    response = client.post_order(order=signed_order, orderType=OrderType.FOK)
                    logger.info(f"Order response: {response}")
                    print(f"Order response: {response}")

                except PolyApiException as e:
                    logger.error(f"PolyApiException during order posting: {str(e)}")
                    print(f"Error posting order: {str(e)}")
                    if hasattr(e, 'error_msg'):
                        print(f"Error message: {e.error_msg}")
                except Exception as e:
                    logger.error(f"Unexpected error during order posting: {str(e)}")
                    print(f"Unexpected error: {str(e)}")
    else:
        logger.info("Not within the allowed time window, skipping buying stocks.")

def selling_stocks(processed_data):
    # Get the current time in Eastern Time
    hour_of_day = datetime.now(pytz.timezone('US/Eastern')).hour
    if hour_of_day >= trading_start and hour_of_day < trading_end:
        logger.info("Starting selling stocks process...")
        selling_stage_1 = []
        token_ids = set()
        for data in processed_data:
            if data['polymarket_probability_to_sell'] >= data['sell_target']:
                selling_stage_1.append(data)
                token_ids.add(data["token"])
                logger.debug(f"Added to selling stage 1: {data}")

        current_positions = pull_user_portfolio(token_ids=token_ids)
        logger.debug(f"Current positions for tokens: {current_positions}")
        print("Current Positions:", current_positions)

        for data in selling_stage_1:
            # if current_positions[data['token']]['amount'] < data['order_min_size']:
            #     logger.info(
            #         f"share size is less than the required amount, we want to sell: {current_positions[data['token']]['amount']}, but min_order_quantity is: {data['order_min_size']}")
            #     continue
            logger.info(
                f"Selling {current_positions[data['token']]['amount']} shares for {data['team']}")
            print(
                f"Selling {current_positions[data['token']]['amount']} shares for {data['team']} at a price of {data['polymarket_probability_to_sell']}")
            signed_order = client.create_order(
                order_args=OrderArgs(
                    price=math.ceil(data['polymarket_probability_to_sell']*100)/100,
                    size=current_positions[data['token']]['amount'],
                    side=SELL,
                    token_id=data["token"]
                )
            )
            try:
                response = client.post_order(order=signed_order, orderType=OrderType.FOK)
                logger.info(f"Successfully sold, Order response: {response}")
            except PolyApiException as e:
                # logger.error(e)
                print(e)
    else:
        logger.info("Not within the allowed time window, skipping selling stocks process.")

def pipeline_trading(week, kelly_multiplier, target_buy_ev, target_sell_ev, time_interval, buy, script_mode):
    bankroll = pull_user_balance()

    pinnacle_params, poly_params = get_mode_params(script_mode=script_mode)

    if not target_buy_ev or not kelly_multiplier or not target_sell_ev or not (1 > float(kelly_multiplier) > 0) or not (
            1 > float(target_sell_ev) > 0) or not (1 > float(target_buy_ev) > 0):
        error = "Please enter a valid target_buy_ev, target_sell_ev and Kelly multiplier."
        print(error)

    # Fetch data from external APIs
    pinnacle_data = pull_from_pinnacle(params=pinnacle_params)
    polymarket_data = pull_from_polymarket_events_api(params=poly_params, week=week)
    logger.info(f"Fetched data from external APIs: pinnacle_data={pinnacle_data}")
    logger.info(f"Fetched data from external APIs: polymarket_data={polymarket_data}")  # Log fetched data

    processed_data = merge_and_process_data(polymarket_data=polymarket_data, pinnacle_data=pinnacle_data, user_inputs={
        "week": week,
        "kelly_multiplier": kelly_multiplier,
        "target_buy_ev": target_buy_ev,
        "target_sell_ev": target_sell_ev,
        "bankroll": bankroll,
    }, for_trade=True)

    logger.info(f"Processed data: {processed_data}")  # Log processed data

    if buy:
        buying_shares(processed_data=processed_data)
        ...
    else:
        ...
        selling_stocks(processed_data=processed_data)

if __name__ == "__main__":
    buy = [True for _ in range(len(SCRIPT_MODES))]  # Start with True
    while True:
        for ind, script_mode in enumerate(SCRIPT_MODES):
            print()
            print()
            print(f"--------------EXECUTING TRADE FOR {script_mode} {'Buy' if buy[ind] else 'Sell'}-----------------")
            try:
                pipeline_trading(
                    week=week[ind],
                    kelly_multiplier=kelly_multiplier[ind],
                    target_buy_ev=target_buy_ev[ind],
                    target_sell_ev=target_sell_ev[ind],
                    time_interval=time_interval,
                    buy=buy[ind],
                    script_mode=script_mode
                )
                # Toggle the value of buy after each iteration
                buy[ind] = not buy[ind]
            except Exception as e:
                logger.error(f"Error occurred: {e}")
                print(f"Error occurred: {e}")
            # Wait for the defined time interval before the next iteration
            time.sleep(time_interval)